In [16]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('gutenberg')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
pip install pattern

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.8/349.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 8.0 MB/s eta 0:00:00
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332702 sha256=ca7c1a41351280770841346fd3d6e15631788994b61cbee8f660b07027b68bee
  Stored in directory: /root/.cache/pip/wheels/d1/8f/40/fe23abd593ef60be

In [1]:
import re
import nltk
def parse_document(document):
  document = re.sub('\n', ' ', document)
  if isinstance(document, str):
    document = document
  elif isinstance(document, unicode):
    return unicodedata.normalize('NFKD', document).encode('ascii','ignore')
  else:
    raise ValueError('Document is not string or unicode!')
  document = document.strip()
  sentences = nltk.sent_tokenize(document)
  sentences = [sentence.strip() for sentence in sentences]
  return sentences

document = '''Café à la mode
            This is cat
            They eat apple'''
print(parse_document(document))

['Café à la mode             This is cat             They eat apple']


In [2]:
#Unescaping HTML Sequence
import html
def unescape_html(text):
    return html.unescape(text)
print(unescape_html("&lt;html&gt;hai&lt;/html&gt;"))

<html>hai</html>


In [19]:
from contractions import CONTRACTION_MAP
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
stopword_list = nltk.corpus.stopwords.words('english')
wnl = WordNetLemmatizer()

def tokenize_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [token.strip() for token in tokens]
  return tokens

def expand_contractions(text, contraction_mapping):
  contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
  flags=re.IGNORECASE|re.DOTALL)
  def expand_match(contraction):
    match = contraction.group(0)
    first_char = match[0]
    expanded_contraction = contraction_mapping.get(match)\
    if contraction_mapping.get(match)\
    else contraction_mapping.get(match.lower())
    expanded_contraction = first_char+expanded_contraction[1:]
    return expanded_contraction
  expanded_text = contractions_pattern.sub(expand_match, text)
  expanded_text = re.sub("'", "", expanded_text)
  return expanded_text

from pattern.en import tag
from nltk.corpus import wordnet as wn
def pos_tag_text(text):
  def penn_to_wn_tags(pos_tag):
    if pos_tag.startswith('J'):
      return wn.ADJ
    elif pos_tag.startswith('V'):
      return wn.VERB
    elif pos_tag.startswith('N'):
      return wn.NOUN
    elif pos_tag.startswith('R'):
      return wn.ADV
    else:
      return None
  tagged_text = tag(text)
  tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
  for word, pos_tag in
  tagged_text]
  return tagged_lower_text


def lemmatize_text(text):
  pos_tagged_text = pos_tag_text(text)
  lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
  else word
  for word, pos_tag in pos_tagged_text]
  lemmatized_text = ' '.join(lemmatized_tokens)
  return lemmatized_text

def remove_special_characters(text):
  tokens = tokenize_text(text)
  pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
  filtered_tokens = filter(None, [pattern.sub('', token) for token in
  tokens])
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

def remove_stopwords(text):
  tokens = tokenize_text(text)
  filtered_tokens = [token for token in tokens if token not in
  stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text


In [20]:
def normalize_corpus(corpus, lemmatize=True, tokenize=False):
  normalized_corpus = []
  for text in corpus:
    text = html.unescape(text)
    text = expand_contractions(text, CONTRACTION_MAP)
    if lemmatize:
      text = lemmatize_text(text)
    else:
      text = text.lower()
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    if tokenize:
      text = tokenize_text(text)
      normalized_corpus.append(text)
    else:
      normalized_corpus.append(text)
  return normalized_corpus

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def build_feature_matrix(documents, feature_type='frequency'):
  feature_type = feature_type.lower().strip()
  if feature_type == 'binary':
    vectorizer = CountVectorizer(binary=True, min_df=1,ngram_range=(1, 1))
  elif feature_type == 'frequency':
    vectorizer = CountVectorizer(binary=False, min_df=1,ngram_range=(1, 1))
  elif feature_type == 'tfidf':
    vectorizer = TfidfVectorizer(min_df=1,ngram_range=(1, 1))
  else:
    raise Exception("Wrong feature type entered. Possible values:'binary', 'frequency', 'tfidf'")
  feature_matrix = vectorizer.fit_transform(documents).astype(float)
  return vectorizer, feature_matrix

In [22]:
from nltk.corpus import gutenberg
import nltk
from operator import itemgetter

alice = gutenberg.sents(fileids='carroll-alice.txt')
alice = [' '.join(ts) for ts in alice]
norm_alice = list(filter(None, normalize_corpus(alice, lemmatize=False)))
print(norm_alice[0])

def flatten_corpus(corpus):
  return ' '.join([document.strip() for document in corpus])

def compute_ngrams(sequence, n):
  return list(zip(*[sequence[index:] for index in range(n)]))

print(compute_ngrams(['He', 'plays','cricket','very','well'],2))

print(compute_ngrams(['He', 'plays','cricket','very','well'],3))

alice adventures wonderland lewis carroll 1865
[('He', 'plays'), ('plays', 'cricket'), ('cricket', 'very'), ('very', 'well')]
[('He', 'plays', 'cricket'), ('plays', 'cricket', 'very'), ('cricket', 'very', 'well')]


In [23]:
def get_top_ngrams(corpus, ngram_val=1, limit=5):
  corpus = flatten_corpus(corpus)
  tokens = nltk.word_tokenize(corpus)
  ngrams = compute_ngrams(tokens, ngram_val)
  ngrams_freq_dist = nltk.FreqDist(ngrams)
  sorted_ngrams_fd = sorted(ngrams_freq_dist.items(), key=itemgetter(1), reverse=True)
  sorted_ngrams = sorted_ngrams_fd[0:limit]
  sorted_ngrams = [(text, freq) for text, freq in sorted_ngrams]
  return sorted_ngrams

In [24]:
print(get_top_ngrams(corpus=norm_alice, ngram_val=2, limit=10))
print(get_top_ngrams(corpus=norm_alice, ngram_val=3, limit=10))

[(('said', 'alice'), 123), (('mock', 'turtle'), 56), (('march', 'hare'), 31), (('said', 'king'), 29), (('thought', 'alice'), 26), (('white', 'rabbit'), 22), (('said', 'hatter'), 22), (('said', 'mock'), 20), (('said', 'caterpillar'), 18), (('said', 'gryphon'), 18)]
[(('said', 'mock', 'turtle'), 20), (('said', 'march', 'hare'), 10), (('poor', 'little', 'thing'), 6), (('little', 'golden', 'key'), 5), (('certainly', 'said', 'alice'), 5), (('white', 'kid', 'gloves'), 5), (('march', 'hare', 'said'), 5), (('mock', 'turtle', 'said'), 5), (('know', 'said', 'alice'), 4), (('might', 'well', 'say'), 4)]


In [25]:
sequence=['I', 'love', 'to', 'sing', 'tamil']
print([sequence[index:] for index in range(3)])
result=list(zip(*[sequence[index:] for index in range(3)]))
print(result)

[['I', 'love', 'to', 'sing', 'tamil'], ['love', 'to', 'sing', 'tamil'], ['to', 'sing', 'tamil']]
[('I', 'love', 'to'), ('love', 'to', 'sing'), ('to', 'sing', 'tamil')]


In [26]:
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
finder = BigramCollocationFinder.from_documents([item.split() for item in norm_alice])
bigram_measures = BigramAssocMeasures()
print('TOP 10 BIGRAMS BASED ON RAW FREQUENCY')
print(finder.nbest(bigram_measures.raw_freq, 10))
print('TOP 10 BIGRAMS BASED ON PMI')
print(finder.nbest(bigram_measures.pmi, 10))

TOP 10 BIGRAMS BASED ON RAW FREQUENCY
[('said', 'alice'), ('mock', 'turtle'), ('march', 'hare'), ('said', 'king'), ('thought', 'alice'), ('said', 'hatter'), ('white', 'rabbit'), ('said', 'mock'), ('said', 'caterpillar'), ('said', 'gryphon')]
TOP 10 BIGRAMS BASED ON PMI
[('abide', 'figures'), ('acceptance', 'elegant'), ('accounting', 'tastes'), ('accustomed', 'usurpation'), ('act', 'crawling'), ('adjourn', 'immediate'), ('adoption', 'energetic'), ('affair', 'trusts'), ('agony', 'terror'), ('alarmed', 'proposal')]


In [27]:
from nltk.collocations import TrigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures
finder = TrigramCollocationFinder.from_documents([item.split() for item in norm_alice])
trigram_measures = TrigramAssocMeasures()
print('TOP 10 TRIGRAMS BASED ON RAW FREQUENCY')
print(finder.nbest(trigram_measures.raw_freq, 10))
print('TOP 10 TRIGRAMS BASED ON PMI')
print(finder.nbest(trigram_measures.pmi, 10))

TOP 10 TRIGRAMS BASED ON RAW FREQUENCY
[('said', 'mock', 'turtle'), ('said', 'march', 'hare'), ('poor', 'little', 'thing'), ('little', 'golden', 'key'), ('march', 'hare', 'said'), ('mock', 'turtle', 'said'), ('white', 'kid', 'gloves'), ('beau', 'ootiful', 'soo'), ('certainly', 'said', 'alice'), ('might', 'well', 'say')]
TOP 10 TRIGRAMS BASED ON PMI
[('accustomed', 'usurpation', 'conquest'), ('adjourn', 'immediate', 'adoption'), ('adoption', 'energetic', 'remedies'), ('ancient', 'modern', 'seaography'), ('apple', 'roast', 'turkey'), ('arithmetic', 'ambition', 'distraction'), ('brother', 'latin', 'grammar'), ('canvas', 'bag', 'tied'), ('cherry', 'tart', 'custard'), ('circle', 'exact', 'shape')]


In [35]:

import itertools
import nltk
from gensim import corpora, models
def get_chunks(sentences, grammar = r'NP: {<DT>? <JJ>* <NN.*>+}'):
  all_chunks = []
  chunker = nltk.chunk.regexp.RegexpParser(grammar)
  for sentence in sentences:
    tagged_sents = nltk.pos_tag_sents([nltk.word_tokenize(sentence)])
    chunks = [chunker.parse(tagged_sent) for tagged_sent in tagged_sents]
    wtc_sents = [nltk.chunk.tree2conlltags(chunk) for chunk in chunks]
    flattened_chunks = list(itertools.chain.from_iterable( wtc_sent for wtc_sent in wtc_sents))
    valid_chunks_tagged = [(status, [wtc for wtc in chunk]) for status, chunk
                                        in itertools.groupby(flattened_chunks, lambda x: x[2] != 'O')]
    valid_chunks = [' '.join(word.lower() for word, tag, chunk in wtc_group
                                            if word.lower() not in stopword_list)
                                                for status, wtc_group in valid_chunks_tagged if status]
    all_chunks.append(valid_chunks)
  return all_chunks

sentences = parse_document(toy_text)
valid_chunks = get_chunks(sentences)
print(valid_chunks)

[['elephants', 'large mammals', 'family elephantidae', 'order proboscidea'], ['species', 'african elephant', 'asian elephant'], ['elephants', 'sub-saharan africa', 'south asia', 'southeast asia'], ['male african elephants', 'extant terrestrial animals'], ['elephants', 'long trunk', 'many purposes', 'breathing', 'water', 'grasping objects'], ['incisors', 'tusks', 'weapons', 'tools', 'objects', 'digging'], ['elephants', 'large ear flaps', 'body temperature'], ['pillar-like legs', 'great weight'], ['african elephants', 'ears', 'backs', 'asian elephants', 'ears', 'convex', 'level backs']]


In [29]:
from scipy.sparse.linalg import svds
def low_rank_svd(matrix, singular_count=2):
  u, s, vt = svds(matrix, k=singular_count)
  return u, s, vt

In [32]:
import numpy as np
toy_text = """
Elephants are large mammals of the family Elephantidae
and the order Proboscidea. Two species are traditionally recognised,
the African elephant and the Asian elephant. Elephants are scattered
throughout sub-Saharan Africa, South Asia, and Southeast Asia. Male
African elephants are the largest extant terrestrial animals. All
elephants have a long trunk used for many purposes,
particularly breathing, lifting water and grasping objects. Their
incisors grow into tusks, which can serve as weapons and as tools
for moving objects and digging. Elephants' large ear flaps help
to control their body temperature. Their pillar-like legs can
carry their great weight. African elephants have larger ears
and concave backs while Asian elephants have smaller ears
and convex or level backs.
"""

num_sentences = 3
num_topics = 3

sentences = parse_document(toy_text)
norm_sentences = normalize_corpus(sentences,lemmatize=True)
print('Total Sentences in Document:', len(norm_sentences))

def lsa_text_summarizer(documents, num_sentences=2, num_topics=2, feature_type='frequency', sv_threshold=0.5):
  vec, dt_matrix = build_feature_matrix(documents, feature_type=feature_type)
  td_matrix = dt_matrix.transpose()
  td_matrix = td_matrix.multiply(td_matrix > 0)
  u, s, vt = low_rank_svd(td_matrix, singular_count=num_topics)
  min_sigma_value = max(s) * sv_threshold
  s[s < min_sigma_value] = 0
  salience_scores = np.sqrt(np.dot(np.square(s), np.square(vt)))
  top_sentence_indices = salience_scores.argsort()[-num_sentences:][::-1]
  top_sentence_indices.sort()
  for index in top_sentence_indices:
    print(sentences[index])

print(lsa_text_summarizer(sentences,3))

Total Sentences in Document: 9
All elephants have a long trunk used for many purposes, particularly breathing, lifting water and grasping objects.
Their incisors grow into tusks, which can serve as weapons and as tools for moving objects and digging.
African elephants have larger ears and concave backs while Asian elephants have smaller ears and convex or level backs.
None
